# **Dependencies**

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs langchain-groq sentence-transformers


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 20.4 MB/s eta 0:00:00
   

In [2]:
import os
from google.colab import userdata
from langchain_groq import ChatGroq

os.environ['GROQ_API_KEY'] = userdata.get('GROK_LLAMA3')
llm = ChatGroq(model_name = 'Gemma2-9b-It')

In [3]:
NEO4J_URI= 'neo4j+s://460c00.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD='oHB7MSwhwRR7jqHZF2OEux9nq7Pa6mvdB9QO1jVw'

In [53]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(
    url = NEO4J_URI,
    username = NEO4J_USERNAME,
    password = NEO4J_PASSWORD
)

# **Data Extraction**

In [5]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query='Elizabeth I').load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [6]:
len(raw_documents)

23

In [7]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size = 512, chunk_overlap = 24)
documents = text_splitter.split_documents(raw_documents[:3])

In [8]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(
    llm = llm,
)

In [9]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [10]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person'), Node(id='Henry Viii', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Edward Vi', type='Person'), Node(id='Mary', type='Person'), Node(id='James Vi Of Scotland', type='Person'), Node(id='William Cecil', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Sir Francis Walsingham', type='Person'), Node(id='Mary, Queen Of Scots', type='Person')], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Henry Viii', type='Person'), type='CHILD'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Anne Boleyn', type='Person'), type='CHILD'), Relationship(source=Node(id='Henry Viii', type='Person'), target=Node(id='Elizabeth I', type='Person'), type='PARENT'), Relationship(source=Node(id='Henry Viii', type='Person'), target=Node(id='Anne Boleyn', type='Person'), type='SPOUSE'), Relationship(source=Node(id='Henry Viii', type='Person'), target=Node

# **Knowledge Graph**

In [11]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel = True,
    include_source = True
)

In [12]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"


In [13]:

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [14]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = NEO4J_URI,
        auth = (NEO4J_USERNAME,
                NEO4J_PASSWORD))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget


In [15]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
from typing import Tuple, List, Optional
from langchain_community.vectorstores import Neo4jVector

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type = 'hybrid',
    node_label = "Document",
    text_node_properties = ['text'],
    embedding_node_property = 'embedding',
    url = NEO4J_URI,
    username = NEO4J_USERNAME,
    password = NEO4J_PASSWORD
)

In [29]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [18]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Entities(BaseModel):
  """Identifying information about entities."""

  names: List[str] = Field(
      ...,
      description = "All the person, organization, or business entities that"
      "appear in the text",
  )

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),

        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        )
    ]


)

In [20]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [21]:
entity_chain.invoke({'question': 'Where was the Amelia Earhert born?'})

Entities(names=['Amelia Earhart'])

In [25]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [26]:

# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [30]:
print(structured_retriever("Who is Elizabeth I?"))

Elizabeth I - CHILD -> Henry Viii
Elizabeth I - CHILD -> Anne Boleyn
Elizabeth I - RELATIVE -> James Vi Of Scotland
Elizabeth I - ADVISOR -> William Cecil
Elizabeth I - ADVISOR -> Sir Francis Walsingham
Elizabeth I - HEIR -> Lady Jane Grey
Henry Viii - PARENT -> Elizabeth I
Edward Vi - SIBLING -> Elizabeth I
Queen Elizabeth I - PARENT -> Henry Viii
Queen Elizabeth I - APPOINTED -> Nicholas Hilliard
Queen Elizabeth I - APPROVED_PORTRAITS -> George Gower
Queen Elizabeth I - PORTRAITIST -> Cornelis Ketel
Queen Elizabeth I - PORTRAITIST -> Federico Zuccaro
Queen Elizabeth I - PORTRAITIST -> Isaac Oliver
Queen Elizabeth I - PORTRAITIST -> Marcus Gheeraerts The Younger


In [44]:
def retriever(question: str):
  print(f"Search query: {question}")
  structured_data = structured_retriever(question)
  unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
  final_data = f"""Structured data:
  {structured_data}
  Unstructured data:
  {"#Document ".join(unstructured_data)}
  """
  return final_data

In [32]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [33]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [49]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

def _format_chat_history(chat_history: list[Tuple[str, str]]) -> List:
  buffer = []
  for human, ai in chat_history:
    buffer.append(HumanMessage(content = human))
    buffer.append(AIMessage(content = ai))
  return buffer

In [40]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llm
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [41]:

template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""


In [42]:
prompt = ChatPromptTemplate.from_template(template)

In [45]:

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)


In [46]:
chain.invoke({"question": "Which house did Elizabeth I belong to?"})

Search query: Which house did Elizabeth I belong to?


'Elizabeth I belonged to the House of Tudor.  \n'

In [51]:

chain.invoke(
    {
        "question": "First Son of Elizabeth 1",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)


Search query: Who was the first son of Elizabeth I? 



'Elizabeth I never had a son, she is known as the "Virgin Queen". \n'